In [1]:
%pip install --upgrade setuptools wheel 

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install wheel pymysql mysql-connector-python sqlalchemy schedule

In [4]:
%pip install git+https://github.com/heebp/pykrx.git


  Cloning https://github.com/heebp/pykrx.git to c:\users\student\appdata\local\temp\pip-req-build-g3uyshux
  Resolved https://github.com/heebp/pykrx.git to commit b0ffe3651dfb7037d5494d03b16a04622aec5a6a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/heebp/pykrx.git 'C:\Users\student\AppData\Local\Temp\pip-req-build-g3uyshux'


In [5]:
from pykrx import stock
from pykrx import bond
import os
from dotenv import load_dotenv
import pandas as pd
import datetime
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")

db_username = DB_USERNAME
db_password = DB_PASSWORD
db_host = DB_HOST
db_name = DB_SCHEME

engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}')

In [6]:
def get_stock_history_columns(table_name):
  metadata = MetaData()
  table = Table(table_name, metadata, autoload_with=engine)
  db_columns = [column.name for column in table.columns]
  return db_columns

In [7]:
def get_price_data(at_date):
  df1 = stock.get_market_ohlcv(at_date, market="KOSPI")
  df2 = stock.get_market_ohlcv(at_date, market="KOSDAQ")

  result = pd.concat([df1, df2], axis=0)

  return result

In [8]:
def get_stock_id_by_short_code():
  Session = sessionmaker(bind=engine)
  session = Session()

  metadata = MetaData()
  stock_table = Table('STOCK', metadata, autoload_with=engine)

  # stock 테이블에서 short_code와 stock_id 조회
  query = session.query(stock_table.c.SHORT_CODE, stock_table.c.STOCK_ID).all()

  session.close()
  
  # short_code와 stock_id를 매핑한 딕셔너리 생성
  return {row.SHORT_CODE: row.STOCK_ID for row in query}


In [9]:
def convert_columns(data):
  data = data.reset_index()

  column_mapping = {
    '티커': 'SHORT_CODE',
    '시가': 'OPEN_PRICE',
    '종가': 'CLOSED_PRICE',
    '고가': 'HIGH_PRICE',
    '저가': 'LOW_PRICE',
  }

  data.rename(columns=column_mapping, inplace=True)
  short_code_to_stock_id_map = get_stock_id_by_short_code()
  data['STOCK_ID'] = data['SHORT_CODE'].map(short_code_to_stock_id_map)
  data['STOCK_ID'] = data['STOCK_ID'].astype(int)
  return data

In [10]:
def fetch_KOSPI_KOSDAQ_price_data():

  price = get_price_data(datetime.datetime.now().strftime("%Y%m%d"))

  price = convert_columns(price)

  return price[['STOCK_ID','SHORT_CODE','CLOSED_PRICE','OPEN_PRICE','HIGH_PRICE','LOW_PRICE']]

In [11]:
def fetch_and_save_data(time):
    # 주식 데이터를 가져오는 함수 호출
    data = fetch_KOSPI_KOSDAQ_price_data()
    print(data)
    data['STOCK_HISTORY_ID'] = time
    data.to_sql(name='STOCK_HISTORY', con=engine, if_exists='append', index=False)
    print("Data saved at:", datetime.datetime.now())


In [12]:

# 주식 개장 시간인지 확인하는 함수
def is_market_open():
    current_time = datetime.datetime.now().time()
    market_open_time = datetime.time(9, 0)  # 09:00 AM
    market_close_time = datetime.time(15, 30)  # 03:30 PM
    return market_open_time <= current_time <= market_close_time


In [13]:
import schedule
import time
import datetime

def stock_market_schedule():
    while True:
        now = datetime.datetime.now()

        # 주식 개장 시간 (9:00 ~ 15:30) 체크
        if now.hour >= 9 and (now.hour < 15 or (now.hour == 15 and now.minute <= 30)):
            
            # 현재 시간이 10분 단위이고, 초가 00초일 때 실행
            if now.minute % 10 == 0 and now.second == 0:
                fetch_and_save_data(now.strftime("%Y%m%d%H%M%S"))        
        # 1초 대기 후 다시 체크
        time.sleep(1)

# 스케줄 시작
stock_market_schedule()

      STOCK_ID SHORT_CODE  CLOSED_PRICE  OPEN_PRICE  HIGH_PRICE  LOW_PRICE
0        28656     095570             0           0           0          0
1        27529     006840             0           0           0          0
2        27890     027410             0           0           0          0
3        29326     282330             0           0           0          0
4        28891     138930             0           0           0          0
...        ...        ...           ...         ...         ...        ...
2707     27845     024060             0           0           0          0
2708     27629     010240             0           0           0          0
2709     29014     189980             0           0           0          0
2710     28041     037440             0           0           0          0
2711     29198     238490             0           0           0          0

[2712 rows x 6 columns]
Data saved at: 2024-10-08 09:00:06.823669
      STOCK_ID SHORT_CODE  CLOSED

Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\sqlalchemy\pool\base.py", line 986, in _finalize_fairy
    fairy._reset(
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\sqlalchemy\pool\base.py", line 1432, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\sqlalchemy\engine\default.py", line 699, in do_rollback
    dbapi_connection.rollback()
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\pymysql\connections.py", line 493, in rollback
    self._read_ok_packet()
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\pymysql\connections.py", line 453, in _read_ok_packet
    pkt = self._read_packet()
          ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\student\Documents\stoc

      STOCK_ID SHORT_CODE  CLOSED_PRICE  OPEN_PRICE  HIGH_PRICE  LOW_PRICE
0        28656     095570          4690        4745        4745       4680
1        27529     006840         13060       13040       13150      12980
2        27890     027410          3555        3560        3565       3540
3        29326     282330        109600      110500      110900     108900
4        28891     138930          9200        9260        9260       9080
...        ...        ...           ...         ...         ...        ...
2707     27845     024060         22000       22650       22900      21500
2708     27629     010240          4885        4870        4890       4870
2709     29014     189980          1869        1829        1994       1828
2710     28041     037440          5190        5170        5240       5140
2711     29198     238490          4715        4725        4900       4680

[2712 rows x 6 columns]
Data saved at: 2024-10-08 11:10:10.031001


Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\sqlalchemy\pool\base.py", line 986, in _finalize_fairy
    fairy._reset(
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\sqlalchemy\pool\base.py", line 1432, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\sqlalchemy\engine\default.py", line 699, in do_rollback
    dbapi_connection.rollback()
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\pymysql\connections.py", line 493, in rollback
    self._read_ok_packet()
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\pymysql\connections.py", line 453, in _read_ok_packet
    pkt = self._read_packet()
          ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\student\Documents\stoc

      STOCK_ID SHORT_CODE  CLOSED_PRICE  OPEN_PRICE  HIGH_PRICE  LOW_PRICE
0        28656     095570          4685        4745        4745       4680
1        27529     006840         13100       13040       13150      12980
2        27890     027410          3555        3560        3565       3540
3        29326     282330        109500      110500      110900     108900
4        28891     138930          9190        9260        9260       9080
...        ...        ...           ...         ...         ...        ...
2707     27845     024060         21850       22650       22900      21500
2708     27629     010240          4875        4870        4890       4870
2709     29014     189980          1861        1829        1994       1828
2710     28041     037440          5180        5170        5240       5140
2711     29198     238490          4730        4725        4900       4680

[2712 rows x 6 columns]
Data saved at: 2024-10-08 11:20:10.205406
      STOCK_ID SHORT_CODE  CLOSED

Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\sqlalchemy\pool\base.py", line 986, in _finalize_fairy
    fairy._reset(
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\sqlalchemy\pool\base.py", line 1432, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\sqlalchemy\engine\default.py", line 699, in do_rollback
    dbapi_connection.rollback()
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\pymysql\connections.py", line 493, in rollback
    self._read_ok_packet()
  File "c:\Users\student\Documents\stock-report-data-generator\stock-info\Lib\site-packages\pymysql\connections.py", line 453, in _read_ok_packet
    pkt = self._read_packet()
          ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\student\Documents\stoc

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'mysql-container.cvswoacu2dtu.us-east-2.rds.amazonaws.com' ([WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)